In [1]:
import pandas as pd
import numpy as np
import datetime
import time
import os
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
start_time = time.time()

size = 10.0;

x_step = 0.2
y_step = 0.2

x_ranges = zip(np.arange(0, size, x_step), np.arange(x_step, size + x_step, x_step));
y_ranges = zip(np.arange(0, size, y_step), np.arange(y_step, size + y_step, y_step));
#print x_ranges
#print y_ranges

print('Calculate hour, weekday, month and year for train and test')
train['hour'] = (train['time']//60)%24+1 # 1 to 24
train['weekday'] = (train['time']//1440)%7+1
train['month'] = (train['time']//43200)%12+1 # rough estimate, month = 30 days
train['year'] = (train['time']//525600)+1 

test['hour'] = (test['time']//60)%24+1 # 1 to 24
test['weekday'] = (test['time']//1440)%7+1
test['month'] = (test['time']//43200)%12+1 # rough estimate, month = 30 days
test['year'] = (test['time']//525600)+1

#print 'shape after time engineering'
#print train.shape
#print test.shape

X_train = train[['x','y','accuracy','time', 'hour', 'weekday', 'month', 'year']];
y_train = train[['place_id']];
X_test = test[['x','y','accuracy','time', 'hour', 'weekday', 'month', 'year']];
X_test_labels = test[['row_id']];

preds_total = pd.DataFrame();
for x_min, x_max in  x_ranges:
    start_time_row = time.time()
    for y_min, y_max in  y_ranges: 
        #start_time_cell = time.time()
        x_max = round(x_max, 4)
        x_min = round(x_min, 4)
        
        y_max = round(y_max, 4)
        y_min = round(y_min, 4)
        
        if x_max == size:
            x_max = x_max + 0.001
            
        if y_max == size:
            y_max = y_max + 0.001
            
        train_grid = train[(train['x'] >= x_min) &
                           (train['x'] < x_max) &
                           (train['y'] >= y_min) &
                           (train['y'] < y_max)]

        test_grid = test[(test['x'] >= x_min) &
                         (test['x'] < x_max) &
                         (test['y'] >= y_min) &
                         (test['y'] < y_max)]
        
        X_train_grid = train_grid[['x','y','accuracy', 'hour', 'weekday', 'month', 'year']];
        y_train_grid = train_grid[['place_id']].values.ravel();
        X_test_grid = test_grid[['x','y','accuracy','hour', 'weekday', 'month', 'year']];
        
        #clf = GradientBoostingClassifier();
        #clf =  LogisticRegression(multi_class='multinomial', solver = 'lbfgs');
        #clf = xgb.XGBClassifier(n_estimators=10);
        clf = RandomForestClassifier(n_estimators = 100, n_jobs = -1,random_state=0);
        clf.fit(X_train_grid, y_train_grid)
        
        preds = dict(zip([el for el in clf.classes_], zip(*clf.predict_proba(X_test_grid))))
        preds = pd.DataFrame.from_dict(preds)

        
        preds['0_'], preds['1_'], preds['2_'] = zip(*preds.apply(lambda x: preds.columns[x.argsort()[::-1][:3]].tolist(), axis=1));
        preds = preds[['0_','1_','2_']];
        preds['row_id'] = test_grid['row_id'].reset_index(drop=True);
        preds_total = pd.concat([preds_total, preds], axis=0);
        #print("Elapsed time cell: %s seconds" % (time.time() - start_time_cell))
    print("{}, Elapsed time row: {} seconds".format(x_min, time.time() - start_time_row))

preds_total['place_id'] = preds_total[['0_', '1_', '2_']].apply(lambda x: ' '.join([str(x1) for x1 in x]), axis=1)
preds_total.drop('0_', axis=1, inplace=True)
preds_total.drop('1_', axis=1, inplace=True)
preds_total.drop('2_', axis=1, inplace=True)
sub_file = os.path.join('rf_submission_' + str(datetime.datetime.now().strftime("%Y-%m-%d-%H-%M")) + '.csv')
preds_total.to_csv(sub_file,index=False)
print("Elapsed time overall: %s seconds" % (time.time() - start_time))

Calculate hour, weekday, month and year for train and test
0.0, Elapsed time row: 117.502125025 seconds
0.2, Elapsed time row: 139.067844868 seconds
0.4, Elapsed time row: 140.713307858 seconds
0.6, Elapsed time row: 146.076611996 seconds
0.8, Elapsed time row: 142.816119194 seconds
1.0, Elapsed time row: 145.774893999 seconds
1.2, Elapsed time row: 154.984300137 seconds
1.4, Elapsed time row: 158.202018023 seconds
1.6, Elapsed time row: 161.066347837 seconds
1.8, Elapsed time row: 162.683815956 seconds
2.0, Elapsed time row: 168.593081951 seconds
2.2, Elapsed time row: 170.873292208 seconds
2.4, Elapsed time row: 180.445017099 seconds
2.6, Elapsed time row: 175.782065153 seconds
2.8, Elapsed time row: 177.274599075 seconds
3.0, Elapsed time row: 168.976785898 seconds
3.2, Elapsed time row: 174.878654003 seconds
3.4, Elapsed time row: 170.382806063 seconds
3.6, Elapsed time row: 171.051793098 seconds
3.8, Elapsed time row: 176.825080156 seconds
4.0, Elapsed time row: 172.232202053 seco